
# 1

Matt/Abi code-snippet to guide one using 'cat_trauma' function    

1.1 Install the relevant packages      

```r
suppressWarnings({
  invisible({
    rm(list = ls())

    if (!require(icdpicr, quietly = TRUE, warn.conflicts = FALSE)) install.packages('icdpicr')
    if (!require(dplyr, quietly = TRUE, warn.conflicts = FALSE)) install.packages('dplyr')
    if (!require(readr, quietly = TRUE, warn.conflicts = FALSE)) install.packages('readr')
    if (!require(tidyr, quietly = TRUE, warn.conflicts = FALSE)) install.packages('tidyr')

    library(icdpicr, quietly = TRUE, warn.conflicts = FALSE)
    library(dplyr, quietly = TRUE, warn.conflicts = FALSE)
    library(readr, quietly = TRUE, warn.conflicts = FALSE)
    library(tidyr, quietly = TRUE, warn.conflicts = FALSE)
  })
})
```

1.2 Get some documentation

```r
ls("package:icdpicr")
```

1.3 Study the syntax

```r
print(cat_trauma)
```

1.4 What is the value of dx_pre?

```r
print(injury)
```

1.5 As such, the syntax is:

```r
# use subset of injury given its size
inj = injury[1:100,1:3]
df_score = cat_trauma(df=inj, dx_pre="dx", icd10=TRUE, i10_iss_method="roc_max_NIS", calc_method=1, verbose=FALSE)
# visualize the output
df_score[1:9,1:9]; df_score[1:9,10:14]; df_score[1:9,15:18]; df_score[1:9,34:35]
```

---

# 2

Calvin Matthew's code-snippet for NHANES Table 1     

```r
install.packages("tableone")
library(tableone)
install.packages("tidyverse")
library(tidyverse)
install.packages("NHANES")
library(NHANES)
install.packages("dplyr")
library(dplyr)
install.packages("nhanesA")
library(nhanesA)

# Creating Dataset With All Years
demo <- bind_rows(nhanes("DEMO"),nhanes("DEMO_B","DEMO_C", "DEMO_D", "DEMO_E", "DEMO_F", "DEMO_G", "DEMO_H", "DEMO_I", "DEMO_J", "DEMO_P"))
# Categorical Data
demo["RIAGENDR"] %>% count(RIAGENDR)
catVars <- c("DMDEDUC2","RIAGENDR", "RIDRETH1")
demo <- subset(demo, RIDAGEYR > 17)
tab <- CreateTableOne(data=demo, factorVars=catVars)

# Sorted Categorical and Continuous Outputs
print(tab$CatTable, showAllLevels=TRUE)
print(tab$ContTable, nonnormal=TRUE)
summary_table <- all_years_data %>% summarize(
MedianAge=median(Age, na.rm=TRUE),
MedianSBP=median(BPSysAve, na.rm=TRUE),
MedianDBP=median(BPDiaAve, na.rm=TRUE),
MedianCreatinine=median(LBDSCRSI, na.rm=TRUE),
MedianBMI=median(BMI, na.rm=TRUE),
MedianHbA1c=median(HbA1c, na.rm=TRUE),
MedianuACR=median(uACR, na.rm=TRUE),
MedianGlucose=median(Pulse, na.rm=TRUE),
FemalePercentage=mean(Sex == "Female", na.rm=TRUE),
HypertensionPercentage=mean(Hypertension == "Yes", na.rm=TRUE),
SmokePercentage=mean(SmokingStatus == "Current", na.rm=TRUE),
RaceEthnicityPercentage=table(RaceEthnicity) / length(RaceEthnicity),
EducationPercentage=table(Education)
)
print(summary_table)

# Cohort Percentage One-by-One
nhanes <- subset(nhanes("DEMO_P"), RIDAGEYR > 17)
divisor <- count(nhanes)
```

# 3

Andrew code for Risk of ESRD in donors    

---

## 3.1 sources

```r

# sources
vars.donor_live.dta <- c("don_id",
              "don_recov_dt",
              "don_age",
              "don_gender",
              "don_race",
              "don_hist_cigarette",
              "don_diab",
              "don_hist_hyperten",
              "don_hgt_cm",
              "don_wgt_kg",
              "don_ki_creat_preop",
              "don_bp_preop_syst",
              "don_bp_preop_diast",
              "don_education",
              "don_relationship_ty"
              )

vars.don_liv_cms.dta <- c("pers_esrd_first_service_dt",
              "pers_ssa_death_dt"
              )

vars.ki.dta <- c("don_id",
              "rec_age_at_tx",
              "rec_dgn"
              )
```

---

## 3.2 packages

```r 
# packages
library(tidyverse)
library(dplyr)
library(haven)
library(tableone)
library(Hmisc)
library(nephro)
library(survival)
library(survminer)
library(lmtest)

# data: File paths go here
# Global path 
global.path <- "~amassie/mock_cdrg/"

# donor_live data goes on the following line
d1 <- read_dta(paste0(global.path,"donor_live.dta")) %>% 
       select(all_of(vars.donor_live.dta), all_of(vars.don_liv_cms.dta))


# tx_ki dataframe here 
d2 <- read_dta(paste0(global.path,"tx_ki.dta")) %>% 
       select(vars.ki.dta)

d <- left_join (d1, d2, by ="don_id")

```

---

## 3.3 variables

```r 
# table1 columns
d$race <-    fct_collapse(as.factor(d$don_race),
  "Asian"="64",
  "Black"="16",
  "Hispanic"="2000",
  "White"=c("8","24","32","40","48","56","72","80","88","96","104","112","128",
            "136","144","152","160","192","200","224","232","248","1024")
)

# table1 rows
d$famRisk <- fct_collapse(as.factor(d$don_relationship_ty),
  "Parent"="1",
  "Child"="2",
  "Twin"="3",
  "Sibling"="4",
  "Other"=c("5","6"),
  "Unrelated"=c("7","8","9","10","11","12","14","15","998","999")
)

d$spouse <- fct_collapse(as.factor(d$don_relationship_ty),
  "Spouse"=c("7","8"),
  "Non-spouse"=c("1","2","3","4","5","6","9","10","11","12","14","15","998","999")
)

d$educ <-    fct_collapse(as.factor(d$don_education),
  "<High School"=c("1","2","3"),
  "Attended College"="4",
  "Graduate"="5",
  "Postgraduate"="6"
)

# replace study-specific codes with NA
d$educ[d$educ==998] <- NA
d$educ[d$educ==996] <- NA

d$year <- as.numeric(substring(d$don_recov_dt,1,4))

d$dm <- NA
d$dm[d$don_diab=="Y"] <- 1
d$dm[d$don_diab=="N"] <- 0

d$htn <-     fct_collapse(as.factor(d$don_hist_hyperten),
   "Hypertension"=c("2","3","4","5"),
   "No Hypertension"=c("1","998")
)
d$htn[d$don_hist_hyperten==998] <- NA

d$cig <- NA
d$cig[d$don_hist_cigarette=="Y"] <- 1
d$cig[d$don_hist_cigarette=="N"] <- 0

d$rec.dgn <-fct_collapse(as.factor(d$rec_dgn),
  "Diabetes"=c("3011","3012","3038","3039","3069","3070","3071","5001","5002","5009"),
  "Hypertension"=c("3034","3040","3050","3066"),
  "Glomerulonephritis"=c("3000","3001","3002","3003","3004","3005","3006","3041",
                         "3042","3043","3051", "3064","3068","3018","3019","3024",
                         "3033","3035","3053","3054","3055","3056","3057"),
  "Cystic Kidney Disease"=c("3008","3028"),
  "Urologic Disease"=c("3013","3014","3015","3031","3032","3036"),
  "Other Diagnosis"=c("3025","3052","3061","3007","3009","3010","3017","3030",
                      "3044","3045","3046","3047","3048","3049","3059","3060","3063",
                      "3073","3016","3020","3021","3022","3023","3058","3062"),
  "Unknown"=c("3026","3027","3029","3072","3074","999")
  )

# egfr formula
d$female <- ifelse(d$don_gender=="F",1,0)
d$male <-  ifelse(d$don_gender=="F",0,1)
d$black <- ifelse(d$don_race==16,1,0)

# egfr estimate is sensitive to this step
#d$don_ki_creat_preop <- ifelse(d$don_ki_creat_preop>3,NA,d$don_ki_creat_preop)
d$k=as.numeric(ifelse(d$don_gender=="F",0.7,0.9))
d$alpha=as.numeric(ifelse(d$don_gender=="F",-0.329,-0.411))

d$egfr <- 141*pmin(d$don_ki_creat_preop/d$k,1,na.rm=TRUE)^d$alpha*
              pmax(d$don_ki_creat_preop/d$k,1,na.rm=TRUE)^-1.209*
                  0.993^d$don_age*
                  1.018^d$female*
                  1.159^d$black

# for now use this!
d$gfr <- CKDEpi.creat(d$don_ki_creat_preop,d$male,d$don_age,d$black)

# implausible = NA
d$gfr[d$gfr>200] <- NA
d$egfr[d$egfr>200] <- NA
d$don_hgt_cm <- ifelse(d$don_hgt_cm>120,d$don_hgt_cm,NA)
d$don_wgt_kg <- ifelse(d$don_wgt_kg<200,d$don_wgt_kg,NA)
d$bmi <- d$don_wgt_kg/(d$don_hgt_cm/100)^2
```

---

## 3.4 table1

```r  
# prep work
listVars <- c("don_age",
              "don_gender",
              "cig",
              "dm",
              "htn",
              "bmi",
              "don_ki_creat_preop",
              "egfr",
              "don_bp_preop_syst",
              "don_bp_preop_diast",
              "year",
              "educ",
              "famRisk",
              "spouse",
              "rec_age_at_tx",
              "rec.dgn"
              )

# subset data
df <- d[,c("race",listVars)]

# continue prep
iqr <-      c("don_age",
              "bmi",
              "egfr",
              "don_ki_creat_preop",
              "don_bp_preop_syst",
              "don_bp_preop_diast",
              "year",
              "rec_age_at_tx"
              )
catVars <- c("don_gender",
              "famRisk",
              "spouse",
              "educ",
              "dm",
              "htn",
              "cig",
              "rec.dgn"
              )

# change order of factors
df$race <-     ordered(df$race,c("Asian","Black","Hispanic","White"))
df$don_gender <-ordered(df$don_gender,c("M","F"))
df$famRisk <-  ordered(df$famRisk,c("Unrelated","Other","Parent","Child",
                                   "Sibling","Twin"))
df$rec.dgn <-  ordered(df$rec.dgn,c("Diabetes","Hypertension","Glomerulonephritis",
                                   "Cystic Kidney Disease","Urologic Disease",
                                   "Other Diagnosis","Unknown"))

# label vars
df <-tibble(df) %>% 
  labelled::set_variable_labels(race = "Race",
    don_age = "Age, y", 
    don_gender = "Women",
    bmi = "Body-Mass Index, Kg/m^2",
    famRisk = "Donor-Recipient Relationship",
    spouse = "Spouse or life partner of recipient",
    educ = "Education",
    dm = "Diabetes",
    htn = "Hypertension",
    egfr = "eGFR, ml/min/1.73m^2",
    don_ki_creat_preop = "Serum Creatinine, mg/dL",
    don_bp_preop_syst = "SBP",
    don_bp_preop_diast = "DBP",
    year = "Year of donation",
    cig = "History of Smoking",
    rec_age_at_tx = "Recipient Age, y",
    rec.dgn = "Recipient Cause of ESRD"
    ) %>%
  labelled::set_value_labels(
    race = c("1" = "Asian", "2" = "Black", "3" = "Hispanic", "4" = "White")
    )

# create table1
table1 <- CreateTableOne(data=df,
                         strata=c("race"),
                         vars=listVars,
                         factorVars=catVars,
                         test=FALSE)
# export to .csv
write.csv(print(table1,
                format=c("p"),
                missing=FALSE,
                nonnormal=iqr,
                varLabels=TRUE,
                dropEqual=TRUE,
                catDigits=1,
                contDigits=1,
                ),"./tableone.csv",row.names=TRUE)
```

---


Asian Americans account for [5.7%](https://minorityhealth.hhs.gov/omh/browse.aspx?lvl=3&lvlid=63) of the nation's population but only `r round(as.numeric(table(df$race)[1])*100/sum(table(df$race)),1)`% of 
the live kidney donor population.

The analysis is run three times: overall, younger than 50, and older than 50


---

## 3.5 datasets

```r 
copy <- df
copy1 <- d
```

---

## 3.6 time-to-event

```r
# dates
df$censor <- as.numeric(as.Date("12/31/20",format="%m/%d/%y"))
df$pers_esrd_first_service_dt <- as.numeric(d$pers_esrd_first_service_dt)
df$pers_ssa_death_dt <- as.numeric(d$pers_ssa_death_dt)
df$don_recov_dt <- as.numeric(d$don_recov_dt)

# events
df$esrd <- ifelse(is.na(d$pers_esrd_first_service_dt),0,1)
df$mort <- ifelse(is.na(d$pers_ssa_death_dt),0,1)
table(df$esrd)
table(df$mort)

df$esrd_dt <- ifelse(is.na(df$pers_esrd_first_service_dt),df$censor,df$pers_esrd_first_service_dt)
df$mort_dt <- ifelse(is.na(df$pers_ssa_death_dt),df$censor,df$pers_ssa_death_dt)

# times
df$fup_esrd <- (df$esrd_dt - df$don_recov_dt)/365.25
df$fup_mort <- (df$mort_dt - df$don_recov_dt)/365.25

# implausible
# highly influential
# no ostriches welcome!
set.seed(053021)
sum(as.numeric(df$fup_esrd<=0))

# adjust p to assess sensitivity
df$esrd <- ifelse(df$fup_esrd<=0,df$esrd*rbinom(1,1,.9),df$esrd) 
df$mort <- ifelse(df$fup_mort<=0,df$mort*rbinom(1,1,.9),df$mort)
table(df$esrd)
table(df$mort)

df$fup_esrd <- ifelse(df$fup_esrd<=0,sample(df$fup_esrd,1),df$fup_esrd)
df$fup_mort <- ifelse(df$fup_mort<=0,sample(df$fup_mort,1),df$fup_mort)
df$end_dt <- min(df$esrd_dt,df$mort_dt)
```

---

## 3.7 kaplan-meier

```r 
# white is reference
df$eth <- d$race

# make unrelated reference
df$fam <- relevel(d$famRisk,c("Unrelated")) 
km = survfit(Surv(fup_esrd,esrd)~eth,data=df)

# black vs. non-black reasonable?
plot(km, conf.int = FALSE, fun = "event", 
     ylim=c(0,.03),
     xlim=c(0,30),
     xlab = "Years", 
     ylab = "Incidence",
     col = c("green","red","blue","violet"), 
     lwd=c(3,3,3,3),las=1,
     lty=c(1,1,2,2),
     cex.axis=1.5,
     cex.lab=1.5)
legend(5,.015,c("Black",
               "White",
               "Asian",
               "Hispanic"),
       lty=c(1,1,2,2),lwd=c(3,3,3,3),col=c("red","green","blue","violet"),cex=1.5,bty="n")

# black vs. non-black reasonable?
fit.eth <- coxph(Surv(fup_esrd,esrd) ~ eth,data=df) 
summary(fit.eth)$coef

# related vs. unrelated reasonable?
fit.fam <- coxph(Surv(fup_esrd,esrd) ~ fam,data=df) 
summary(fit.fam)$coef

# strong a priori reasonable?
fit.full <- coxph(Surv(fup_esrd,esrd) ~ eth*fam,data=df) 
summary(fit.full)$coef

# strong a priori reasonable?
fit.null <- coxph(Surv(fup_esrd,esrd) ~ eth+fam,data=df) 
summary(fit.null)$coef

# model with interaction is better fit 
lrt <- lmtest::lrtest(fit.full,fit.null); lrt
chi <- round(as.matrix(lrt)[2,4],3); chi
pval <- round(as.matrix(lrt)[2,5],3); pval
```

---

Where $Y$ is log of the relative hazards and $\beta_i$ (where $i=1,2,...,23$ ) are regression coefficients:

$Y=\beta_1ethBlack+\beta_2ethAsian+\beta_3ethHispanic+\beta_4famParent+\beta_5famChild+\beta_6famTwin+\beta_7famSibling+\beta_8famOther+\beta_9ethBlack:famParent+\beta_{10}ethAsian:famParent+\beta_{11}ethHispanic:famParent+\beta_{12}ethBlack:famChild+\beta_{13}ethAsian:famChild+\beta_{14}ethHispanic:famChild+\beta_{15}ethBlack:famTwin+\beta_{16}ethAsian:famTwin+\beta_{17}ethHsipanic:famTwin+\beta_{18}ethBlack:famSibling+\beta_{19}ethAsian:famSibling+\beta_{20}ethHispanic:famSibling+\beta_{21}ethBlack:famOther+\beta_{22}ethAsian:famOther+\beta_{23}ethHispanic:famOther$

---

## 3.8 cox model

```r
# basic output
beta.fit.full <- coefficients(fit.full)
vcov.fit.full <- vcov(fit.full)
write.csv(print(beta.fit.full
                ),"./betafamRisk.csv",row.names=TRUE)
write.csv(print(vcov.fit.full
                ),"./vcovfamRisk.csv",row.names=TRUE)

## unrelated: white is reference
## speaks to contextual effects or linkage disequilibrium?
# black: b1
Abu = c(1,rep(0,22))
bu = as.numeric(t(Abu)%*%coefficients(fit.full))
se.bu = sqrt(as.numeric(Abu)%*%vcov(fit.full)%*%Abu)
exp(bu); exp(bu+c(-1,1)*1.96*se.bu)

# asian: b2
Aau = c(0,1,rep(0,21))
au = as.numeric(t(Aau)%*%coefficients(fit.full))
se.au = sqrt(as.numeric(Aau)%*%vcov(fit.full)%*%Aau)
exp(au); exp(au+c(-1,1)*1.96*se.au)

# hispanic: b3
Ahu = c(0,0,1,rep(0,20))
hu = as.numeric(t(Ahu)%*%coefficients(fit.full))
se.hu = sqrt(as.numeric(Ahu)%*%vcov(fit.full)%*%Ahu)
exp(hu); exp(hu+c(-1,1)*1.96*se.hu)

## white
# white twin: b6
Awt = c(rep(0,5),1,rep(0,17))
wt = as.numeric(t(Awt)%*%coefficients(fit.full))
se.wt = sqrt(as.numeric(Awt)%*%vcov(fit.full)%*%Awt)
exp(wt); exp(wt+c(-1,1)*1.96*se.wt)

# white sibling: b7
Aws = c(rep(0,6),1,rep(0,16))
ws = as.numeric(t(Aws)%*%coefficients(fit.full))
se.ws = sqrt(as.numeric(Aws)%*%vcov(fit.full)%*%Aws)
exp(ws); exp(ws+c(-1,1)*1.96*se.ws)

# white child: b5
Awc = c(rep(0,6),1,rep(0,16))
wc = as.numeric(t(Awc)%*%coefficients(fit.full))
se.wc = sqrt(as.numeric(Awc)%*%vcov(fit.full)%*%Awc)
exp(wc); exp(wc+c(-1,1)*1.96*se.wc)

# white parent: b4
Awp = c(rep(0,3),1,rep(0,19))
wp = as.numeric(t(Awp)%*%coefficients(fit.full))
se.wp = sqrt(as.numeric(Awp)%*%vcov(fit.full)%*%Awp)
exp(wp); exp(wp+c(-1,1)*1.96*se.wp)

# white other: b8
Awo = c(rep(0,7),1,rep(0,15))
wo = as.numeric(t(Awo)%*%coefficients(fit.full))
se.wo = sqrt(as.numeric(Awo)%*%vcov(fit.full)%*%Awo)
exp(wo); exp(wp+c(-1,1)*1.96*se.wo)

## black
# black unrelated: b1
Abu = c(1,rep(0,22))
bu = as.numeric(t(Abu)%*%coefficients(fit.full))
se.bu = sqrt(as.numeric(Abu)%*%vcov(fit.full)%*%Abu)
exp(bu); exp(bu+c(-1,1)*1.96*se.bu)

# black twin: b1 + b6 + b15
Abt = c(rep(0,5),1,rep(0,8),1,rep(0,8))
bt = as.numeric(t(Abt)%*%coefficients(fit.full))
se.bt = sqrt(as.numeric(Abt)%*%vcov(fit.full)%*%Abt)
exp(bt); exp(bt+c(-1,1)*1.96*se.bt)

# black sibling: b1 + b7 + b18
Abs = c(rep(0,6),1,rep(0,10),1,rep(0,5))
bs = as.numeric(t(Abs)%*%coefficients(fit.full))
se.bs = sqrt(as.numeric(Abs)%*%vcov(fit.full)%*%Abs)
exp(bs); exp(bs+c(-1,1)*1.96*se.bs)

# black child: b1 + b5 + b12
Abc = c(rep(0,4),1,rep(0,6),1,rep(0,11))
bc = as.numeric(t(Abc)%*%coefficients(fit.full))
se.bc = sqrt(as.numeric(Abc)%*%vcov(fit.full)%*%Abc)
exp(bc); exp(bc+c(-1,1)*1.96*se.bc)

# black parent: b1 + b4 + b9
Abp = c(rep(0,3),1,rep(0,4),1,rep(0,14))
bp = as.numeric(t(Abp)%*%coefficients(fit.full))
se.bp = sqrt(as.numeric(Abp)%*%vcov(fit.full)%*%Abp)
exp(bp); exp(bp+c(-1,1)*1.96*se.bp)

# black other: b1 + b8 + b21
Abo = c(rep(0,7),1,rep(0,12),1,rep(0,2))
bo = as.numeric(t(Abo)%*%coefficients(fit.full))
se.bo = sqrt(as.numeric(Abo)%*%vcov(fit.full)%*%Abo)
exp(bo); exp(bo+c(-1,1)*1.96*se.bo)

## hispanic
# hispanic unrelated: b3
Ahu = c(rep(0,2),1,rep(0,20))
hu = as.numeric(t(Ahu)%*%coefficients(fit.full))
se.hu = sqrt(as.numeric(Ahu)%*%vcov(fit.full)%*%Ahu)
exp(hu); exp(hu+c(-1,1)*1.96*se.hu)

# hispanic twin: b3 + b6 + b17
Aht = c(rep(0,5),1,rep(0,10),1,rep(0,6))
ht = as.numeric(t(Aht)%*%coefficients(fit.full))
se.ht = sqrt(as.numeric(Aht)%*%vcov(fit.full)%*%Aht)
exp(ht); exp(ht+c(-1,1)*1.96*se.ht)

# hispanic sibling: b3 + b7 + b20
Ahs = c(rep(0,6),1,rep(0,12),1,rep(0,3))
hs = as.numeric(t(Ahs)%*%coefficients(fit.full))
se.hs = sqrt(as.numeric(Ahs)%*%vcov(fit.full)%*%Ahs)
exp(hs); exp(hs+c(-1,1)*1.96*se.hs)

# hispanic child: b3 + b5 + b14
Ahc = c(rep(0,4),1,rep(0,8),1,rep(0,9))
hc = as.numeric(t(Ahc)%*%coefficients(fit.full))
se.hc = sqrt(as.numeric(Ahc)%*%vcov(fit.full)%*%Ahc)
exp(hc); exp(hc+c(-1,1)*1.96*se.hc)

# hispanic parent: b3 + b4 + b11
Ahp = c(rep(0,3),1,rep(0,6),1,rep(0,12))
hp = as.numeric(t(Ahp)%*%coefficients(fit.full))
se.hp = sqrt(as.numeric(Ahp)%*%vcov(fit.full)%*%Ahp)
exp(hp); exp(hp+c(-1,1)*1.96*se.hp)

# hispanic other: b3 + b8 + b23
Aho = c(rep(0,7),1,rep(0,14),1)
ho = as.numeric(t(Aho)%*%coefficients(fit.full))
se.ho = sqrt(as.numeric(Aho)%*%vcov(fit.full)%*%Aho)
exp(ho); exp(ho+c(-1,1)*1.96*se.ho)

## asian
# asian unrelated: b2
Aau = c(0,1,rep(0,21))
au = as.numeric(t(Aau)%*%coefficients(fit.full))
se.au = sqrt(as.numeric(Aau)%*%vcov(fit.full)%*%Aau)
exp(au); exp(au+c(-1,1)*1.96*se.au)

# asian twin: b2 + b6 + b16
Aat = c(rep(0,5),1,rep(0,9),1,rep(0,7))
at = as.numeric(t(Aat)%*%coefficients(fit.full))
se.at = sqrt(as.numeric(Aat)%*%vcov(fit.full)%*%Aat)
exp(at); exp(at+c(-1,1)*1.96*se.at)

# asian sibling: b2 + b7 + b19
Aas = c(rep(0,6),1,rep(0,11),1,rep(0,4))
as = as.numeric(t(Aas)%*%coefficients(fit.full))
se.as = sqrt(as.numeric(Aas)%*%vcov(fit.full)%*%Aas)
exp(as); exp(as+c(-1,1)*1.96*se.as)

# asian child: b2 + b5 + b13
Aac = c(rep(0,4),1,rep(0,8),1,rep(0,9))
ac = as.numeric(t(Aac)%*%coefficients(fit.full))
se.ac = sqrt(as.numeric(Aac)%*%vcov(fit.full)%*%Aac)
exp(ac); exp(ac+c(-1,1)*1.96*se.ac)

# asian parent: b2 + b4 + b10
Aap = c(rep(0,3),1,rep(0,5),1,rep(0,13))
ap = as.numeric(t(Aap)%*%coefficients(fit.full))
se.ap = sqrt(as.numeric(Aap)%*%vcov(fit.full)%*%Aap)
exp(ap); exp(ap+c(-1,1)*1.96*se.ap)

# asian other: b2 + b8 + b22
Aao = c(rep(0,7),1,rep(0,13),1,0)
ao = as.numeric(t(Aao)%*%coefficients(fit.full))
se.ao = sqrt(as.numeric(Aao)%*%vcov(fit.full)%*%Aao)
exp(ao); exp(ao+c(-1,1)*1.96*se.ao)
```

---

## 3.9 fig1 

```r
# hr (95% CI)

fig1.df <- data.frame(x=2:21,
           hr=c(exp(wo),
                exp(wp),
                exp(wc),
                exp(ws),
                exp(wt),
                exp(bo),
                exp(bp),
                exp(bc),
                exp(bs),
                exp(bt),
                exp(ho),
                exp(hp),
                exp(hc),
                exp(hs),
                exp(ht),
                exp(ao),
                exp(ap),
                exp(ac),
                exp(as),
                exp(at)),
           lb.hr=c(exp(wo-1.96*se.wo),
                   exp(wp-1.96*se.wp),
                   exp(wc-1.96*se.wc),
                   exp(ws-1.96*se.ws),
                   exp(wt-1.96*se.wt),
                   exp(bo-1.96*se.bo),
                   exp(bp-1.96*se.bp),
                   exp(bc-1.96*se.bc),
                   exp(bs-1.96*se.bs),
                   exp(bt-1.96*se.bt),
                   exp(ho-1.96*se.ho),
                   exp(hp-1.96*se.hp),
                   exp(hc-1.96*se.hc),
                   exp(hs-1.96*se.hs),
                   exp(ht-1.96*se.ht),
                   exp(ao-1.96*se.ao),
                   exp(ap-1.96*se.ap),
                   exp(ac-1.96*se.ac),
                   exp(as-1.96*se.as),
                   exp(at-1.96*se.at)),
           ub.hr=c(exp(wo+1.96*se.wo),
                   exp(wp+1.96*se.wp),
                   exp(wc+1.96*se.wc),
                   exp(ws+1.96*se.ws),
                   exp(wt+1.96*se.wt),
                   exp(bo+1.96*se.bo),
                   exp(bp+1.96*se.bp),
                   exp(bc+1.96*se.bc),
                   exp(bs+1.96*se.bs),
                   exp(bt+1.96*se.bt),
                   exp(ho+1.96*se.ho),
                   exp(hp+1.96*se.hp),
                   exp(hc+1.96*se.hc),
                   exp(hs+1.96*se.hs),
                   exp(ht+1.96*se.ht),
                   exp(ao+1.96*se.ao),
                   exp(ap+1.96*se.ap),
                   exp(ac+1.96*se.ac),
                   exp(as+1.96*se.as),
                   exp(at+1.96*se.at))
           )

fig1.df <- rbind(1,fig1.df)
fig1.df$race <- c(rep(1,6),rep(2,5),rep(3,5),rep(4,5))
fig1.df$x <- ordered(fig1.df$x, levels=c(1:21), 
                        labels=c("Unr",
                                 "Oth",
                                 "Par",
                                 "Off",
                                 "Sib",
                                 "Twi",
                                 "Oth ",
                                 "Par ",
                                 "Off ",
                                 "Sib ",
                                 "Twi ",
                                 "Oth  ",
                                 "Par  ",
                                 "Off  ",
                                 "Sib  ",
                                 "Twi  ",
                                 "Oth   ",
                                 "Par   ",
                                 "Off   ",
                                 "Sib   ",
                                 "Twi   "
                                 )
                     )
ggplot(data=fig1.df, aes(x = factor(x), y=hr, color=as.factor(race))) +
  theme_bw() +
  geom_point() +
  geom_hline(yintercept = 1, color="red") +
  labs(x=" ", y="Hazard Ratios (95% CI)", 
       title = " ") +
  scale_color_manual(name="Race",
                     values=c("1" = "2", "2"="3", "3"="4", "4"="5"),
                     labels=c("White", "Black","Hispanic","Asian")) + 
  geom_errorbar(width=.1, aes(ymin=lb.hr, ymax=ub.hr)) +
  scale_y_log10(breaks=c(.01,.1,1,10,100,1000,10000),limits=c(.01,10000),labels=c("0.01","0.1","1","10","100","1000","10000"))
```

---

## 3.10 fig2

```r
km = survfit(Surv(fup_esrd,esrd)~race,data=df)
ggsurvplot(
  km, 
  data=df, 
  fun="event",
  risk.table=T,
  test.for.trend=T,
  conf.int=F,
  xlim=c(0,30),
  ylim=c(0,.03),
  linetype=c(1,2,3,4),
  break.time.by=5,
  ggtheme=theme_minimal(),
  lty=c(1:4),
  lwd=3,
  risk.table.y.text.col=T,
  risk.table.y.text=F,
  surv.plot.height=.9,
  risk.table.height=.3,
  censor=F
)
```

---

## 3.11 time-to-event

Young Donors Analysis


```r
# dates
df <- copy[copy$don_age<50,]
d <- copy1[copy1$don_age<50,]
df$censor <- as.numeric(as.Date("12/31/20",format="%m/%d/%y"))
df$pers_esrd_first_service_dt <- as.numeric(d$pers_esrd_first_service_dt)
df$pers_ssa_death_dt <- as.numeric(d$pers_ssa_death_dt)
df$don_recov_dt <- as.numeric(d$don_recov_dt)

# events
df$esrd <- ifelse(is.na(d$pers_esrd_first_service_dt),0,1)
df$mort <- ifelse(is.na(d$pers_ssa_death_dt),0,1)
table(df$esrd)
table(df$mort)

df$esrd_dt <- ifelse(is.na(df$pers_esrd_first_service_dt),df$censor,df$pers_esrd_first_service_dt)
df$mort_dt <- ifelse(is.na(df$pers_ssa_death_dt),df$censor,df$pers_ssa_death_dt)

# times
df$fup_esrd <- (df$esrd_dt - df$don_recov_dt)/365.25
df$fup_mort <- (df$mort_dt - df$don_recov_dt)/365.25

# implausible
# highly influential
# no ostriches welcome!
set.seed(053021)
sum(as.numeric(df$fup_esrd<=0))

# adjust p to assess sensitivity
df$esrd <- ifelse(df$fup_esrd<=0,df$esrd*rbinom(1,1,.9),df$esrd) 
df$mort <- ifelse(df$fup_mort<=0,df$mort*rbinom(1,1,.9),df$mort)
table(df$esrd)
table(df$mort)

df$fup_esrd <- ifelse(df$fup_esrd<=0,sample(df$fup_esrd,1),df$fup_esrd)
df$fup_mort <- ifelse(df$fup_mort<=0,sample(df$fup_mort,1),df$fup_mort)
df$end_dt <- min(df$esrd_dt,df$mort_dt)
```

## 3.12 kaplan-meier

```r
# white is reference
df$eth <- d$race

# make unrelated reference
df$fam <- relevel(d$famRisk,c("Unrelated")) 
km = survfit(Surv(fup_esrd,esrd)~eth,data=df)

# black vs. non-black reasonable?
plot(km, conf.int = FALSE, fun = "event", 
     ylim=c(0,.02),
     xlim=c(0,30),
     xlab = "Years", 
     ylab = "Incidence",
     col = c("green","red","blue","violet"), 
     lwd=c(3,3,3,3),las=1,
     lty=c(1,1,2,2),
     cex.axis=1.5,
     cex.lab=1.5)
legend(5,.015,c("Black",
               "White",
               "Asian",
               "Hispanic"),
       lty=c(1,1,2,2),lwd=c(3,3,3,3),col=c("red","green","blue","violet"),cex=1.5,bty="n")

# black vs. non-black reasonable?
fit.eth <- coxph(Surv(fup_esrd,esrd) ~ eth,data=df) 
summary(fit.eth)$coef

# related vs. unrelated reasonable?
fit.fam <- coxph(Surv(fup_esrd,esrd) ~ fam,data=df) 
summary(fit.fam)$coef


# strong a priori reasonable?
fit.full <- coxph(Surv(fup_esrd,esrd) ~ eth*fam+don_age+don_gender,data=df) 
summary(fit.full)$coef

# strong a priori reasonable?
fit.null <- coxph(Surv(fup_esrd,esrd) ~ eth+fam+don_age+don_gender,data=df) 
summary(fit.null)$coef

# model with interaction is better fit 
lrt <- lmtest::lrtest(fit.full,fit.null); lrt
chi <- round(as.matrix(lrt)[2,4],3); chi
pval <- round(as.matrix(lrt)[2,5],3); pval
```

---

## 3.13 cox model

```r
# basic output
beta.fit.full <- coefficients(fit.full)
vcov.fit.full <- vcov(fit.full)
write.csv(print(beta.fit.full
                ),"./betafamRisk.csv",row.names=TRUE)
write.csv(print(vcov.fit.full
                ),"./vcovfamRisk.csv",row.names=TRUE)
coef <- coefficients(fit.full)
coef[is.na(coef)] <- 0 
## unrelated: white is reference
## speaks to contextual effects or linkage disequilibrium?
## b9 and b10 are age and sex terms
# black: b1
Abu = c(1,rep(0,24))
bu = as.numeric(t(Abu)%*%coef)
se.bu = sqrt(as.numeric(Abu)%*%vcov(fit.full)%*%Abu)
exp(bu); exp(bu+c(-1,1)*1.96*se.bu)

# asian: b2
Aau = c(0,1,rep(0,23))
au = as.numeric(t(Aau)%*%coef)
se.au = sqrt(as.numeric(Aau)%*%vcov(fit.full)%*%Aau)
exp(au); exp(au+c(-1,1)*1.96*se.au)

# hispanic: b3
Ahu = c(0,0,1,rep(0,22))
hu = as.numeric(t(Ahu)%*%coef)
se.hu = sqrt(as.numeric(Ahu)%*%vcov(fit.full)%*%Ahu)
exp(hu); exp(hu+c(-1,1)*1.96*se.hu)

## white
# white twin: b6
Awt = c(rep(0,5),1,rep(0,19))
wt = as.numeric(t(Awt)%*%coef)
se.wt = sqrt(as.numeric(Awt)%*%vcov(fit.full)%*%Awt)
exp(wt); exp(wt+c(-1,1)*1.96*se.wt)

# white sibling: b7
Aws = c(rep(0,6),1,rep(0,18))
ws = as.numeric(t(Aws)%*%coef)
se.ws = sqrt(as.numeric(Aws)%*%vcov(fit.full)%*%Aws)
exp(ws); exp(ws+c(-1,1)*1.96*se.ws)

# white child: b5
Awc = c(rep(0,4),1,rep(0,20))
wc = as.numeric(t(Awc)%*%coef)
se.wc = sqrt(as.numeric(Awc)%*%vcov(fit.full)%*%Awc)
exp(wc); exp(wc+c(-1,1)*1.96*se.wc)

# white parent: b4
Awp = c(rep(0,3),1,rep(0,21))
wp = as.numeric(t(Awp)%*%coef)
se.wp = sqrt(as.numeric(Awp)%*%vcov(fit.full)%*%Awp)
exp(wp); exp(wp+c(-1,1)*1.96*se.wp)

# white other: b8
Awo = c(rep(0,7),1,rep(0,17))
wo = as.numeric(t(Awo)%*%coef)
se.wo = sqrt(as.numeric(Awo)%*%vcov(fit.full)%*%Awo)
exp(wo); exp(wp+c(-1,1)*1.96*se.wo)

## black
# black unrelated: b1
Abu = c(1,rep(0,24))
bu = as.numeric(t(Abu)%*%coef)
se.bu = sqrt(as.numeric(Abu)%*%vcov(fit.full)%*%Abu)
exp(bu); exp(bu+c(-1,1)*1.96*se.bu)

# black twin: b1 + b6 + b17
Abt = c(rep(0,5),1,rep(0,10),1,rep(0,8))
bt = as.numeric(t(Abt)%*%coef)
se.bt = sqrt(as.numeric(Abt)%*%vcov(fit.full)%*%Abt)
exp(bt); exp(bt+c(-1,1)*1.96*se.bt)

# black sibling: b1 + b7 + b20
Abs = c(rep(0,6),1,rep(0,12),1,rep(0,5))
bs = as.numeric(t(Abs)%*%coef)
se.bs = sqrt(as.numeric(Abs)%*%vcov(fit.full)%*%Abs)
exp(bs); exp(bs+c(-1,1)*1.96*se.bs)

# black child: b1 + b5 + b14
Abc = c(rep(0,4),1,rep(0,8),1,rep(0,11))
bc = as.numeric(t(Abc)%*%coef)
se.bc = sqrt(as.numeric(Abc)%*%vcov(fit.full)%*%Abc)
exp(bc); exp(bc+c(-1,1)*1.96*se.bc)

# black parent: b1 + b4 + b11
Abp = c(rep(0,3),1,rep(0,6),1,rep(0,14))
bp = as.numeric(t(Abp)%*%coef)
se.bp = sqrt(as.numeric(Abp)%*%vcov(fit.full)%*%Abp)
exp(bp); exp(bp+c(-1,1)*1.96*se.bp)

# black other: b1 + b8 + b23
Abo = c(rep(0,7),1,rep(0,14),1,rep(0,2))
bo = as.numeric(t(Abo)%*%coef)
se.bo = sqrt(as.numeric(Abo)%*%vcov(fit.full)%*%Abo)
exp(bo); exp(bo+c(-1,1)*1.96*se.bo)

## hispanic
# hispanic unrelated: b3
Ahu = c(rep(0,2),1,rep(0,22))
hu = as.numeric(t(Ahu)%*%coef)
se.hu = sqrt(as.numeric(Ahu)%*%vcov(fit.full)%*%Ahu)
exp(hu); exp(hu+c(-1,1)*1.96*se.hu)

# hispanic twin: b3 + b6 + b19
Aht = c(rep(0,5),1,rep(0,12),1,rep(0,6))
ht = as.numeric(t(Aht)%*%coef)
se.ht = sqrt(as.numeric(Aht)%*%vcov(fit.full)%*%Aht)
exp(ht); exp(ht+c(-1,1)*1.96*se.ht)

# hispanic sibling: b3 + b7 + b22
Ahs = c(rep(0,6),1,rep(0,14),1,rep(0,3))
hs = as.numeric(t(Ahs)%*%coef)
se.hs = sqrt(as.numeric(Ahs)%*%vcov(fit.full)%*%Ahs)
exp(hs); exp(hs+c(-1,1)*1.96*se.hs)

# hispanic child: b3 + b5 + b16
Ahc = c(rep(0,4),1,rep(0,10),1,rep(0,9))
hc = as.numeric(t(Ahc)%*%coef)
se.hc = sqrt(as.numeric(Ahc)%*%vcov(fit.full)%*%Ahc)
exp(hc); exp(hc+c(-1,1)*1.96*se.hc)

# hispanic parent: b3 + b4 + b13
Ahp = c(rep(0,3),1,rep(0,8),1,rep(0,12))
hp = as.numeric(t(Ahp)%*%coef)
se.hp = sqrt(as.numeric(Ahp)%*%vcov(fit.full)%*%Ahp)
exp(hp); exp(hp+c(-1,1)*1.96*se.hp)

# hispanic other: b3 + b8 + b25
Aho = c(rep(0,7),1,rep(0,16),1)
ho = as.numeric(t(Aho)%*%coef)
se.ho = sqrt(as.numeric(Aho)%*%vcov(fit.full)%*%Aho)
exp(ho); exp(ho+c(-1,1)*1.96*se.ho)

## asian
# asian unrelated: b2
Aau = c(0,1,rep(0,23))
au = as.numeric(t(Aau)%*%coef)
se.au = sqrt(as.numeric(Aau)%*%vcov(fit.full)%*%Aau)
exp(au); exp(au+c(-1,1)*1.96*se.au)

# asian twin: b2 + b6 + b18
Aat = c(rep(0,5),1,rep(0,11),1,rep(0,7))
at = as.numeric(t(Aat)%*%coef)
se.at = sqrt(as.numeric(Aat)%*%vcov(fit.full)%*%Aat)
exp(at); exp(at+c(-1,1)*1.96*se.at)

# asian sibling: b2 + b7 + b21
Aas = c(rep(0,6),1,rep(0,13),1,rep(0,4))
as = as.numeric(t(Aas)%*%coef)
se.as = sqrt(as.numeric(Aas)%*%vcov(fit.full)%*%Aas)
exp(as); exp(as+c(-1,1)*1.96*se.as)

# asian child: b2 + b5 + b15
Aac = c(rep(0,4),1,rep(0,9),1,rep(0,10))
ac = as.numeric(t(Aac)%*%coef)
se.ac = sqrt(as.numeric(Aac)%*%vcov(fit.full)%*%Aac)
exp(ac); exp(ac+c(-1,1)*1.96*se.ac)

# asian parent: b2 + b4 + b12
Aap = c(rep(0,3),1,rep(0,7),1,rep(0,13))
ap = as.numeric(t(Aap)%*%coef)
se.ap = sqrt(as.numeric(Aap)%*%vcov(fit.full)%*%Aap)
exp(ap); exp(ap+c(-1,1)*1.96*se.ap)

# asian other: b2 + b8 + b24
Aao = c(rep(0,7),1,rep(0,15),1,0)
ao = as.numeric(t(Aao)%*%coef)
se.ao = sqrt(as.numeric(Aao)%*%vcov(fit.full)%*%Aao)
exp(ao); exp(ao+c(-1,1)*1.96*se.ao)
```

---

## 3.14 fig1

```r
# hr (95% CI)

fig1.df <- data.frame(x=2:21,
           hr=c(exp(wo),
                exp(wp),
                exp(wc),
                exp(ws),
                exp(wt),
                exp(bo),
                exp(bp),
                exp(bc),
                exp(bs),
                exp(bt),
                exp(ho),
                exp(hp),
                exp(hc),
                exp(hs),
                exp(ht),
                exp(ao),
                exp(ap),
                exp(ac),
                exp(as),
                exp(at)),
           lb.hr=c(exp(wo-1.96*se.wo),
                   exp(wp-1.96*se.wp),
                   exp(wc-1.96*se.wc),
                   exp(ws-1.96*se.ws),
                   exp(wt-1.96*se.wt),
                   exp(bo-1.96*se.bo),
                   exp(bp-1.96*se.bp),
                   exp(bc-1.96*se.bc),
                   exp(bs-1.96*se.bs),
                   exp(bt-1.96*se.bt),
                   exp(ho-1.96*se.ho),
                   exp(hp-1.96*se.hp),
                   exp(hc-1.96*se.hc),
                   exp(hs-1.96*se.hs),
                   exp(ht-1.96*se.ht),
                   exp(ao-1.96*se.ao),
                   exp(ap-1.96*se.ap),
                   exp(ac-1.96*se.ac),
                   exp(as-1.96*se.as),
                   exp(at-1.96*se.at)),
           ub.hr=c(exp(wo+1.96*se.wo),
                   exp(wp+1.96*se.wp),
                   exp(wc+1.96*se.wc),
                   exp(ws+1.96*se.ws),
                   exp(wt+1.96*se.wt),
                   exp(bo+1.96*se.bo),
                   exp(bp+1.96*se.bp),
                   exp(bc+1.96*se.bc),
                   exp(bs+1.96*se.bs),
                   exp(bt+1.96*se.bt),
                   exp(ho+1.96*se.ho),
                   exp(hp+1.96*se.hp),
                   exp(hc+1.96*se.hc),
                   exp(hs+1.96*se.hs),
                   exp(ht+1.96*se.ht),
                   exp(ao+1.96*se.ao),
                   exp(ap+1.96*se.ap),
                   exp(ac+1.96*se.ac),
                   exp(as+1.96*se.as),
                   exp(at+1.96*se.at))
           )

fig1.df <- rbind(1,fig1.df)
fig1.df$race <- c(rep(1,6),rep(2,5),rep(3,5),rep(4,5))
fig1.df$x <- ordered(fig1.df$x, levels=c(1:21), 
                        labels=c("Unr",
                                 "Oth",
                                 "Par",
                                 "Off",
                                 "Sib",
                                 "Twi",
                                 "Oth ",
                                 "Par ",
                                 "Off ",
                                 "Sib ",
                                 "Twi ",
                                 "Oth  ",
                                 "Par  ",
                                 "Off  ",
                                 "Sib  ",
                                 "Twi  ",
                                 "Oth   ",
                                 "Par   ",
                                 "Off   ",
                                 "Sib   ",
                                 "Twi   "
                                 )
                     )
ggplot(data=fig1.df, aes(x = factor(x), y=hr, color=as.factor(race))) +
  theme_bw() +
  geom_point() +
  geom_hline(yintercept = 1, color="red") +
  labs(x=" ", y="Hazard Ratios (95% CI)", 
       title = " ") +
  scale_color_manual(name="Race",
                     values=c("1" = "2", "2"="3", "3"="4", "4"="5"),
                     labels=c("White", "Black","Hispanic","Asian")) + 
  geom_errorbar(width=.1, aes(ymin=lb.hr, ymax=ub.hr)) +
  scale_y_log10(breaks=c(.01,.1,1,10,100,1000,10000),limits=c(.01,10000),labels=c("0.01","0.1","1","10","100","1000","10000"))

# using previous paper format graphs, separated by race as well 
ggplot(fig1.df[1:6,], aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="White Donors")

tmp1 <- rbind(c(0,1,1,1,2),fig1.df[7:11,])
tmp1[1,1] <- "Unr"

ggplot(tmp1, aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="Black Donors")

tmp2 <- rbind(c(0,1,1,1,3),fig1.df[12:16,])
tmp2[1,1] <- "Unr"
ggplot(tmp2, aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="Hispanic Donors")

tmp3 <- rbind(c(0,1,1,1,4),fig1.df[17:21,])
tmp3[1,1] <- "Unr"
ggplot(tmp3, aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="Asian Donors")
```

---

## 3.15 fig3

```r

km = survfit(Surv(fup_esrd,esrd)~race,data=df)
ggsurvplot(
  km, 
  data=df, 
  fun="event",
  risk.table=T,
  test.for.trend=T,
  conf.int=F,
  xlim=c(0,30),
  ylim=c(0,.02),
  linetype=c(1,2,3,4),
  break.time.by=5,
  ggtheme=theme_minimal(),
  lty=c(1:4),
  lwd=3,
  risk.table.y.text.col=T,
  risk.table.y.text=F,
  surv.plot.height=.9,
  risk.table.height=.3,
  censor=F
)
```

--- 

## 3.16 fig4

Older Donors

```r
# dates
df <- copy[copy$don_age>=50,]
d <- copy1[copy1$don_age>=50,]
df$censor <- as.numeric(as.Date("12/31/20",format="%m/%d/%y"))
df$pers_esrd_first_service_dt <- as.numeric(d$pers_esrd_first_service_dt)
df$pers_ssa_death_dt <- as.numeric(d$pers_ssa_death_dt)
df$don_recov_dt <- as.numeric(d$don_recov_dt)

# events
df$esrd <- ifelse(is.na(d$pers_esrd_first_service_dt),0,1)
df$mort <- ifelse(is.na(d$pers_ssa_death_dt),0,1)
table(df$esrd)
table(df$mort)

df$esrd_dt <- ifelse(is.na(df$pers_esrd_first_service_dt),df$censor,df$pers_esrd_first_service_dt)
df$mort_dt <- ifelse(is.na(df$pers_ssa_death_dt),df$censor,df$pers_ssa_death_dt)

# times
df$fup_esrd <- (df$esrd_dt - df$don_recov_dt)/365.25
df$fup_mort <- (df$mort_dt - df$don_recov_dt)/365.25

# implausible
# highly influential
# no ostriches welcome!
set.seed(053021)
sum(as.numeric(df$fup_esrd<=0))

# adjust p to assess sensitivity
df$esrd <- ifelse(df$fup_esrd<=0,df$esrd*rbinom(1,1,.9),df$esrd) 
df$mort <- ifelse(df$fup_mort<=0,df$mort*rbinom(1,1,.9),df$mort)
table(df$esrd)
table(df$mort)

df$fup_esrd <- ifelse(df$fup_esrd<=0,sample(df$fup_esrd,1),df$fup_esrd)
df$fup_mort <- ifelse(df$fup_mort<=0,sample(df$fup_mort,1),df$fup_mort)
df$end_dt <- min(df$esrd_dt,df$mort_dt)
```

---

## 3.17 fig4 kaplan-meier

```r
# white is reference
df$eth <- d$race

# make unrelated reference
df$fam <- relevel(d$famRisk,c("Unrelated")) 
km = survfit(Surv(fup_esrd,esrd)~eth,data=df)

# black vs. non-black reasonable?
plot(km, conf.int = FALSE, fun = "event", 
     ylim=c(0,.02),
     xlim=c(0,30),
     xlab = "Years", 
     ylab = "Incidence",
     col = c("green","red","blue","violet"), 
     lwd=c(3,3,3,3),las=1,
     lty=c(1,1,2,2),
     cex.axis=1.5,
     cex.lab=1.5)
legend(5,.015,c("Black",
               "White",
               "Asian",
               "Hispanic"),
       lty=c(1,1,2,2),lwd=c(3,3,3,3),col=c("red","green","blue","violet"),cex=1.5,bty="n")

# black vs. non-black reasonable?
fit.eth <- coxph(Surv(fup_esrd,esrd) ~ eth,data=df) 
summary(fit.eth)$coef

# related vs. unrelated reasonable?
fit.fam <- coxph(Surv(fup_esrd,esrd) ~ fam,data=df) 
summary(fit.fam)$coef

# over versus under 50 reasonable?
df <- df %>% mutate(age50=(don_age>=50))
fit.age <- coxph(Surv(fup_esrd, esrd)~age50, data=df)
summary(fit.age)$coef

# strong a priori reasonable?
fit.full <- coxph(Surv(fup_esrd,esrd) ~ eth*fam+don_age+don_gender,data=df) 
summary(fit.full)$coef

# strong a priori reasonable?
fit.null <- coxph(Surv(fup_esrd,esrd) ~ eth+fam+don_age+don_gender,data=df) 
summary(fit.null)$coef

# model with interaction is better fit 
lrt <- lmtest::lrtest(fit.full,fit.null); lrt
chi <- round(as.matrix(lrt)[2,4],3); chi
pval <- round(as.matrix(lrt)[2,5],3); pval
```

---

## 3.18 lincom

```r

# basic output
beta.fit.full <- coefficients(fit.full)
vcov.fit.full <- vcov(fit.full)
write.csv(print(beta.fit.full
                ),"./betafamRisk.csv",row.names=TRUE)
write.csv(print(vcov.fit.full
                ),"./vcovfamRisk.csv",row.names=TRUE)
coef <- coefficients(fit.full)
coef[is.na(coef)] <- 0 
## unrelated: white is reference
## speaks to contextual effects or linkage disequilibrium?
## b9 and b10 are age and sex terms
# black: b1
Abu = c(1,rep(0,24))
bu = as.numeric(t(Abu)%*%coef)
se.bu = sqrt(as.numeric(Abu)%*%vcov(fit.full)%*%Abu)
exp(bu); exp(bu+c(-1,1)*1.96*se.bu)

# asian: b2
Aau = c(0,1,rep(0,23))
au = as.numeric(t(Aau)%*%coef)
se.au = sqrt(as.numeric(Aau)%*%vcov(fit.full)%*%Aau)
exp(au); exp(au+c(-1,1)*1.96*se.au)

# hispanic: b3
Ahu = c(0,0,1,rep(0,22))
hu = as.numeric(t(Ahu)%*%coef)
se.hu = sqrt(as.numeric(Ahu)%*%vcov(fit.full)%*%Ahu)
exp(hu); exp(hu+c(-1,1)*1.96*se.hu)

## white
# white twin: b6
Awt = c(rep(0,5),1,rep(0,19))
wt = as.numeric(t(Awt)%*%coef)
se.wt = sqrt(as.numeric(Awt)%*%vcov(fit.full)%*%Awt)
exp(wt); exp(wt+c(-1,1)*1.96*se.wt)

# white sibling: b7
Aws = c(rep(0,6),1,rep(0,18))
ws = as.numeric(t(Aws)%*%coef)
se.ws = sqrt(as.numeric(Aws)%*%vcov(fit.full)%*%Aws)
exp(ws); exp(ws+c(-1,1)*1.96*se.ws)

# white child: b5
Awc = c(rep(0,4),1,rep(0,20))
wc = as.numeric(t(Awc)%*%coef)
se.wc = sqrt(as.numeric(Awc)%*%vcov(fit.full)%*%Awc)
exp(wc); exp(wc+c(-1,1)*1.96*se.wc)

# white parent: b4
Awp = c(rep(0,3),1,rep(0,21))
wp = as.numeric(t(Awp)%*%coef)
se.wp = sqrt(as.numeric(Awp)%*%vcov(fit.full)%*%Awp)
exp(wp); exp(wp+c(-1,1)*1.96*se.wp)

# white other: b8
Awo = c(rep(0,7),1,rep(0,17))
wo = as.numeric(t(Awo)%*%coef)
se.wo = sqrt(as.numeric(Awo)%*%vcov(fit.full)%*%Awo)
exp(wo); exp(wp+c(-1,1)*1.96*se.wo)

## black
# black unrelated: b1
Abu = c(1,rep(0,24))
bu = as.numeric(t(Abu)%*%coef)
se.bu = sqrt(as.numeric(Abu)%*%vcov(fit.full)%*%Abu)
exp(bu); exp(bu+c(-1,1)*1.96*se.bu)

# black twin: b1 + b6 + b17
Abt = c(rep(0,5),1,rep(0,10),1,rep(0,8))
bt = as.numeric(t(Abt)%*%coef)
se.bt = sqrt(as.numeric(Abt)%*%vcov(fit.full)%*%Abt)
exp(bt); exp(bt+c(-1,1)*1.96*se.bt)

# black sibling: b1 + b7 + b20
Abs = c(rep(0,6),1,rep(0,12),1,rep(0,5))
bs = as.numeric(t(Abs)%*%coef)
se.bs = sqrt(as.numeric(Abs)%*%vcov(fit.full)%*%Abs)
exp(bs); exp(bs+c(-1,1)*1.96*se.bs)

# black child: b1 + b5 + b14
Abc = c(rep(0,4),1,rep(0,8),1,rep(0,11))
bc = as.numeric(t(Abc)%*%coef)
se.bc = sqrt(as.numeric(Abc)%*%vcov(fit.full)%*%Abc)
exp(bc); exp(bc+c(-1,1)*1.96*se.bc)

# black parent: b1 + b4 + b11
Abp = c(rep(0,3),1,rep(0,6),1,rep(0,14))
bp = as.numeric(t(Abp)%*%coef)
se.bp = sqrt(as.numeric(Abp)%*%vcov(fit.full)%*%Abp)
exp(bp); exp(bp+c(-1,1)*1.96*se.bp)

# black other: b1 + b8 + b23
Abo = c(rep(0,7),1,rep(0,14),1,rep(0,2))
bo = as.numeric(t(Abo)%*%coef)
se.bo = sqrt(as.numeric(Abo)%*%vcov(fit.full)%*%Abo)
exp(bo); exp(bo+c(-1,1)*1.96*se.bo)

## hispanic
# hispanic unrelated: b3
Ahu = c(rep(0,2),1,rep(0,22))
hu = as.numeric(t(Ahu)%*%coef)
se.hu = sqrt(as.numeric(Ahu)%*%vcov(fit.full)%*%Ahu)
exp(hu); exp(hu+c(-1,1)*1.96*se.hu)

# hispanic twin: b3 + b6 + b19
Aht = c(rep(0,5),1,rep(0,12),1,rep(0,6))
ht = as.numeric(t(Aht)%*%coef)
se.ht = sqrt(as.numeric(Aht)%*%vcov(fit.full)%*%Aht)
exp(ht); exp(ht+c(-1,1)*1.96*se.ht)

# hispanic sibling: b3 + b7 + b22
Ahs = c(rep(0,6),1,rep(0,14),1,rep(0,3))
hs = as.numeric(t(Ahs)%*%coef)
se.hs = sqrt(as.numeric(Ahs)%*%vcov(fit.full)%*%Ahs)
exp(hs); exp(hs+c(-1,1)*1.96*se.hs)

# hispanic child: b3 + b5 + b16
Ahc = c(rep(0,4),1,rep(0,10),1,rep(0,9))
hc = as.numeric(t(Ahc)%*%coef)
se.hc = sqrt(as.numeric(Ahc)%*%vcov(fit.full)%*%Ahc)
exp(hc); exp(hc+c(-1,1)*1.96*se.hc)

# hispanic parent: b3 + b4 + b13
Ahp = c(rep(0,3),1,rep(0,8),1,rep(0,12))
hp = as.numeric(t(Ahp)%*%coef)
se.hp = sqrt(as.numeric(Ahp)%*%vcov(fit.full)%*%Ahp)
exp(hp); exp(hp+c(-1,1)*1.96*se.hp)

# hispanic other: b3 + b8 + b25
Aho = c(rep(0,7),1,rep(0,16),1)
ho = as.numeric(t(Aho)%*%coef)
se.ho = sqrt(as.numeric(Aho)%*%vcov(fit.full)%*%Aho)
exp(ho); exp(ho+c(-1,1)*1.96*se.ho)

## asian
# asian unrelated: b2
Aau = c(0,1,rep(0,23))
au = as.numeric(t(Aau)%*%coef)
se.au = sqrt(as.numeric(Aau)%*%vcov(fit.full)%*%Aau)
exp(au); exp(au+c(-1,1)*1.96*se.au)

# asian twin: b2 + b6 + b18
Aat = c(rep(0,5),1,rep(0,11),1,rep(0,7))
at = as.numeric(t(Aat)%*%coef)
se.at = sqrt(as.numeric(Aat)%*%vcov(fit.full)%*%Aat)
exp(at); exp(at+c(-1,1)*1.96*se.at)

# asian sibling: b2 + b7 + b21
Aas = c(rep(0,6),1,rep(0,13),1,rep(0,4))
as = as.numeric(t(Aas)%*%coef)
se.as = sqrt(as.numeric(Aas)%*%vcov(fit.full)%*%Aas)
exp(as); exp(as+c(-1,1)*1.96*se.as)

# asian child: b2 + b5 + b15
Aac = c(rep(0,4),1,rep(0,9),1,rep(0,10))
ac = as.numeric(t(Aac)%*%coef)
se.ac = sqrt(as.numeric(Aac)%*%vcov(fit.full)%*%Aac)
exp(ac); exp(ac+c(-1,1)*1.96*se.ac)

# asian parent: b2 + b4 + b12
Aap = c(rep(0,3),1,rep(0,7),1,rep(0,13))
ap = as.numeric(t(Aap)%*%coef)
se.ap = sqrt(as.numeric(Aap)%*%vcov(fit.full)%*%Aap)
exp(ap); exp(ap+c(-1,1)*1.96*se.ap)

# asian other: b2 + b8 + b24
Aao = c(rep(0,7),1,rep(0,15),1,0)
ao = as.numeric(t(Aao)%*%coef)
se.ao = sqrt(as.numeric(Aao)%*%vcov(fit.full)%*%Aao)
exp(ao); exp(ao+c(-1,1)*1.96*se.ao)
```

---

## 3.19 hr, 95%ci

```r

fig1.df <- data.frame(x=2:21,
           hr=c(exp(wo),
                exp(wp),
                exp(wc),
                exp(ws),
                exp(wt),
                exp(bo),
                exp(bp),
                exp(bc),
                exp(bs),
                exp(bt),
                exp(ho),
                exp(hp),
                exp(hc),
                exp(hs),
                exp(ht),
                exp(ao),
                exp(ap),
                exp(ac),
                exp(as),
                exp(at)),
           lb.hr=c(exp(wo-1.96*se.wo),
                   exp(wp-1.96*se.wp),
                   exp(wc-1.96*se.wc),
                   exp(ws-1.96*se.ws),
                   exp(wt-1.96*se.wt),
                   exp(bo-1.96*se.bo),
                   exp(bp-1.96*se.bp),
                   exp(bc-1.96*se.bc),
                   exp(bs-1.96*se.bs),
                   exp(bt-1.96*se.bt),
                   exp(ho-1.96*se.ho),
                   exp(hp-1.96*se.hp),
                   exp(hc-1.96*se.hc),
                   exp(hs-1.96*se.hs),
                   exp(ht-1.96*se.ht),
                   exp(ao-1.96*se.ao),
                   exp(ap-1.96*se.ap),
                   exp(ac-1.96*se.ac),
                   exp(as-1.96*se.as),
                   exp(at-1.96*se.at)),
           ub.hr=c(exp(wo+1.96*se.wo),
                   exp(wp+1.96*se.wp),
                   exp(wc+1.96*se.wc),
                   exp(ws+1.96*se.ws),
                   exp(wt+1.96*se.wt),
                   exp(bo+1.96*se.bo),
                   exp(bp+1.96*se.bp),
                   exp(bc+1.96*se.bc),
                   exp(bs+1.96*se.bs),
                   exp(bt+1.96*se.bt),
                   exp(ho+1.96*se.ho),
                   exp(hp+1.96*se.hp),
                   exp(hc+1.96*se.hc),
                   exp(hs+1.96*se.hs),
                   exp(ht+1.96*se.ht),
                   exp(ao+1.96*se.ao),
                   exp(ap+1.96*se.ap),
                   exp(ac+1.96*se.ac),
                   exp(as+1.96*se.as),
                   exp(at+1.96*se.at))
           )

fig1.df <- rbind(1,fig1.df)
fig1.df$race <- c(rep(1,6),rep(2,5),rep(3,5),rep(4,5))
fig1.df$x <- ordered(fig1.df$x, levels=c(1:21), 
                        labels=c("Unr",
                                 "Oth",
                                 "Par",
                                 "Off",
                                 "Sib",
                                 "Twi",
                                 "Oth ",
                                 "Par ",
                                 "Off ",
                                 "Sib ",
                                 "Twi ",
                                 "Oth  ",
                                 "Par  ",
                                 "Off  ",
                                 "Sib  ",
                                 "Twi  ",
                                 "Oth   ",
                                 "Par   ",
                                 "Off   ",
                                 "Sib   ",
                                 "Twi   "
                                 )
                     )
ggplot(data=fig1.df, aes(x = factor(x), y=hr, color=as.factor(race))) +
  theme_bw() +
  geom_point() +
  geom_hline(yintercept = 1, color="red") +
  labs(x=" ", y="Hazard Ratios (95% CI)", 
       title = " ") +
  scale_color_manual(name="Race",
                     values=c("1" = "2", "2"="3", "3"="4", "4"="5"),
                     labels=c("White", "Black","Hispanic","Asian")) + 
  geom_errorbar(width=.1, aes(ymin=lb.hr, ymax=ub.hr)) +
  scale_y_log10(breaks=c(.01,.1,1,10,100,1000,10000),limits=c(.01,10000),labels=c("0.01","0.1","1","10","100","1000","10000"))

# using previous paper format graphs, separated by race as well 
ggplot(fig1.df[1:6,], aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="White Donors")

tmp1 <- rbind(c(0,1,1,1,2),fig1.df[7:11,])
tmp1[1,1] <- "Unr"

ggplot(tmp1, aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="Black Donors")

tmp2 <- rbind(c(0,1,1,1,3),fig1.df[12:16,])
tmp2[1,1] <- "Unr"
ggplot(tmp2, aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="Hispanic Donors")

tmp3 <- rbind(c(0,1,1,1,4),fig1.df[17:21,])
tmp3[1,1] <- "Unr"
ggplot(tmp3, aes(x,hr))+geom_errorbar(aes(ymin=lb.hr, ymax=ub.hr), color="red", width=0.3)+geom_point(color="blue")+scale_y_log10(breaks=c(.01,.1,1,10,100,1000),limits=c(.01,1000),labels=c("0.01","0.1","1","10","100","1000"))+theme_classic()+theme(panel.border = element_rect(color="black", fill=NA),strip.background = element_rect(color = NA, fill = NA))+geom_hline(yintercept=1, linetype="dashed", color = "darkgreen")+labs(y = "Adjusted 30-Year Risk of ESKD (95% CI)", x = "Donor-Recipient Relationship", title="Asian Donors")
```

--- 

## 3.20 kaplan-meier

```r

km = survfit(Surv(fup_esrd,esrd)~race,data=df)
ggsurvplot(
  km, 
  data=df, 
  fun="event",
  risk.table=T,
  test.for.trend=T,
  conf.int=F,
  xlim=c(0,30),
  ylim=c(0,.02),
  linetype=c(1,2,3,4),
  break.time.by=5,
  ggtheme=theme_minimal(),
  lty=c(1:4),
  lwd=3,
  risk.table.y.text.col=T,
  risk.table.y.text=F,
  surv.plot.height=.9,
  risk.table.height=.3,
  censor=F
)
```

## 3.21 coef matrix

In [1]:
import pandas as pd

data = {
    'Variable': [
        'ethBlack', 'ethAsian', 'ethHispanic', 'famParent', 'famChild', 'famTwin', 'famSibling', 'famOther',
        'don_age', 'don_gender.L', 'ethBlack:famParent', 'ethAsian:famParent', 'ethHispanic:famParent',
        'ethBlack:famChild', 'ethAsian:famChild', 'ethHispanic:famChild', 'ethBlack:famTwin', 'ethAsian:famTwin',
        'ethHispanic:famTwin', 'ethBlack:famSibling', 'ethAsian:famSibling', 'ethHispanic:famSibling',
        'ethBlack:famOther', 'ethAsian:famOther', 'ethHispanic:famOther'
    ],
    'Value': [
        1.0151953246669, -14.927497085093, 0.678461896702298, 0.230265573210636, 0.013544913171065, 1.42910461013563,
        0.236455951477178, -0.787491191503819, 0.0119317639310039, -0.572389580257542, -0.86969428217781,
        14.3877734454682, -0.993232170840791, -16.038225913901, -0.123974478022701, -15.6236161172565, -17.3373645602685,
        "NA", -17.2130352804811, -1.08195857557585, 15.880675396788, -0.445355654043098, -15.1325387892128,
        0.712239764066844, 0.734976112544702
    ]
}

df = pd.DataFrame(data)

print(df)


                  Variable      Value
0                 ethBlack   1.015195
1                 ethAsian -14.927497
2              ethHispanic   0.678462
3                famParent   0.230266
4                 famChild   0.013545
5                  famTwin   1.429105
6               famSibling   0.236456
7                 famOther  -0.787491
8                  don_age   0.011932
9             don_gender.L   -0.57239
10      ethBlack:famParent  -0.869694
11      ethAsian:famParent  14.387773
12   ethHispanic:famParent  -0.993232
13       ethBlack:famChild -16.038226
14       ethAsian:famChild  -0.123974
15    ethHispanic:famChild -15.623616
16        ethBlack:famTwin -17.337365
17        ethAsian:famTwin         NA
18     ethHispanic:famTwin -17.213035
19     ethBlack:famSibling  -1.081959
20     ethAsian:famSibling  15.880675
21  ethHispanic:famSibling  -0.445356
22       ethBlack:famOther -15.132539
23       ethAsian:famOther    0.71224
24    ethHispanic:famOther   0.734976


## 3.22 var-cov matrix

In [17]:
import pandas as pd
from io import StringIO

data_str = """
	ethBlack	ethAsian	ethHispanic	famParent	famChild	famTwin	famSibling	famOther	don_age	don_gender.L	ethBlack:famParent	ethAsian:famParent	ethHispanic:famParent	ethBlack:famChild	don_age	ethHispanic:famChild	ethBlack:famTwin	ethAsian:famTwin	ethHispanic:famTwin	ethBlack:famSibling	ethAsian:famSibling	ethHispanic:famSibling	ethBlack:famOther	ethAsian:famOther	ethHispanic:famOther
ethBlack	0.190894678788476	0.0235558442160578	0.0239652420030226	0.0239074469413327	0.0250593904018294	0.0233161243880576	0.0243570674309782	0.02406557675074	0.000291018909049624	0.00075699762938253	-0.190533006549935	-0.0237027802326494	-0.023861631979757	-0.190720406678491	0.000291018909049624	-0.0238165315451206	-0.190130381503623	0	-0.0232010584696114	-0.190747726705487	-0.0237227293836337	-0.0238509353218037	-0.190543539419183	-0.0235982539171211	-0.023603936601252
ethAsian	0.0235558442160578	2776034.58680885	0.0237356706031963	0.0234854690671213	0.0231070683959841	0.0238888385802423	0.0233354802479866	0.0235445958797368	-0.00016356422103387	-0.000812957663761613	-0.0237003819249422	-2776034.586553	-0.0237383547297809	-0.0237071458502807	-0.00016356422103387	-0.0237715473888845	-0.0234052052999598	0	-0.0240387707656515	-0.0236110008310505	-2776034.5866118	-0.0237364359515111	-0.0236701519763141	-2776034.58679018	-0.0239230334423816
ethHispanic	0.0239652420030226	0.0237356706031963	0.22388527128733	0.0238408190283083	0.0243077120769637	0.023547048506404	0.0240139930927071	0.0238886758141502	0.000117122753680552	0.000131034364789064	-0.0238300060610353	-0.0238089559773688	-0.22384370428551	-0.0239179904234862	0.000117122753680552	-0.223777638581397	-0.0233999424957336	0	-0.223601144766981	-0.0239250194373203	-0.023801014687455	-0.223813013498558	-0.0238263306272248	-0.023789277122574	-0.223726741307308
famParent	0.0239074469413327	0.0234854690671213	0.0238408190283083	0.0473145436335008	0.0229531992196133	0.0262866443094213	0.0259868633880186	0.0252961945643431	-7.54195985158145E-05	0.000657761593986339	-0.0445237889252271	-0.04511521465306	-0.0443046060551221	-0.023667970863374	-7.54195985158145E-05	-0.0230511871237024	-0.0290640218005166	0	-0.0265801278647533	-0.0245339728562016	-0.0247176644360182	-0.0242234510144319	-0.0253119509898178	-0.0232965565986291	-0.024071231298798
famChild	0.0250593904018294	0.0231070683959841	0.0243077120769637	0.0229531992196133	1.02831451320895	0.0208464039762603	0.024963700147173	0.0241349367091637	0.00101216030782392	0.000765896440579043	-0.0235725950836856	-0.0231342294716064	-0.0237076209578611	-1.02474653853163	0.00101216030782392	-1.02413235518908	-0.019824787967461	0	-0.0207864581296927	-0.0244398779038476	-0.0233200894472744	-0.0236168646444364	-0.0233180123120797	-0.0235839907800947	-0.0228839936455257
famTwin	0.0233161243880576	0.0238888385802423	0.023547048506404	0.0262866443094213	0.0208464039762603	1.02765522379058	0.0243231825450455	0.024526536951529	-0.000623282544920257	0.00251077402793043	-0.0245861027101174	-0.0247170475190836	-0.0243157299671054	-0.023042680871504	-0.000623282544920257	-0.023191391522418	-1.03065789841987	0	-1.02729018632717	-0.0238941784465175	-0.0243571582473462	-0.0244530227229242	-0.0252047524365899	-0.023290926232266	-0.0246502916961579
famSibling	0.0243570674309782	0.0233354802479866	0.0240139930927071	0.0259868633880186	0.024963700147173	0.0243231825450455	0.0506724574453309	0.0249179845266547	0.000341694088942578	0.000549775954737047	-0.0244065128866379	-0.0245662718716258	-0.0243299724457504	-0.0240873704094057	0.000341694088942578	-0.023433388655679	-0.0258671400374308	0	-0.0244515600249059	-0.0495008143165027	-0.0491843526099389	-0.0489881758273534	-0.0245653620210343	-0.0233673440735018	-0.0236162617067214
famOther	0.02406557675074	0.0235445958797368	0.0238886758141502	0.0252961945643431	0.0241349367091637	0.024526536951529	0.0249179845266547	0.357855860962318	0.000125416706482289	0.000532437226052883	-0.0242187760911225	-0.0242973279239917	-0.0241471643798098	-0.0238901592233782	0.000125416706482289	-0.0235507325572539	-0.0258196381038334	0	-0.0245280303417383	-0.0241941326313094	-0.0240633404940337	-0.0239708466462066	-0.357703907374572	-0.356812464911102	-0.357118256510536
don_age	0.000291018909049624	-0.00016356422103387	0.000117122753680552	-7.54195985158145E-05	0.00101216030782392	-0.000623282544920257	0.000341694088942578	0.000125416706482289	0.000234428401327097	5.61503624781432E-05	2.87001202691717E-05	0.00011165199169082	-3.69716615405781E-06	-0.000222428903854332	0.000234428401327097	-4.03088988333006E-05	0.000777672635218292	0	0.000616274373564922	-0.000164805987751014	8.14979501336502E-05	4.56138907343494E-05	8.1919139857354E-05	5.99870523943372E-05	0.000213025091058376
don_gender.L	0.00075699762938253	-0.000812957663761613	0.000131034364789064	0.000657761593986339	0.000765896440579043	0.00251077402793043	0.000549775954737047	0.000532437226052883	5.61503624781432E-05	0.0121380387964427	-0.000818108948751681	5.9093048021296E-05	-0.000210126093038624	0.000708597768668398	5.61503624781432E-05	-0.000122702251291008	-0.00534909868135995	0	-0.000296504219099567	-0.000650651023982381	0.000314268952068898	-0.00141223691620152	-0.00173540146186277	0.00158139474718642	-0.00068275005754803
ethBlack:famParent	-0.190533006549935	-0.0237003819249422	-0.0238300060610353	-0.0445237889252271	-0.0235725950836856	-0.0245861027101174	-0.0244065128866379	-0.0242187760911225	2.87001202691717E-05	-0.000818108948751681	0.41046175900419	0.0439398391063759	0.043682278787996	0.190408873947731	2.87001202691717E-05	0.0235975600548969	0.192051357036436	0	0.0245953362800284	0.190719242042659	0.0240756536327409	0.0239985854006052	0.191001954079834	0.0235903636163333	0.0239408946123176
ethAsian:famParent	-0.0237027802326494	-2776034.586553	-0.0238089559773688	-0.04511521465306	-0.0231342294716064	-0.0247170475190836	-0.0245662718716258	-0.0242973279239917	0.00011165199169082	5.9093048021296E-05	0.0439398391063759	2776035.60724489	0.0438340149246634	0.0236869717108996	0.00011165199169082	0.0234589470915579	0.0252237394017576	0	0.0250557367097241	0.024094026959306	2776034.58712189	0.0239570786920573	0.0244546720071311	2776034.58655996	0.023972746891028
ethHispanic:famParent	-0.023861631979757	-0.0237383547297809	-0.22384370428551	-0.0443046060551221	-0.0237076209578611	-0.0243157299671054	-0.0243299724457504	-0.0241471643798098	-3.69716615405781E-06	-0.000210126093038624	0.043682278787996	0.0438340149246634	0.493641902513275	0.0238025974429005	-3.69716615405781E-06	0.223600325177085	0.024785467926547	0	0.224453588118488	0.0240096334900731	0.024031499303689	0.22391408434107	0.0241756199051724	0.0237168621460648	0.22386607780078
ethBlack:famChild	-0.190720406678491	-0.0237071458502807	-0.0239179904234862	-0.023667970863374	-1.02474653853163	-0.023042680871504	-0.0240873704094057	-0.0238901592233782	-0.000222428903854332	0.000708597768668398	0.190408873947731	0.0236869717108996	0.0238025974429005	21840922.1470653	-0.000222428903854332	1.02384293304659	0.189351128638998	0	0.0232592877699353	0.190601161706669	0.023740903059757	0.0236703449794517	0.190300753822968	0.0238345023166807	0.0235737583740995
ethAsian:famChild	-0.0240203554988254	-2776034.58634793	-0.023813756118959	-0.0249343329510328	-1.02433508772954	-0.0238808597020131	-0.0249045043977197	-0.0245260268243064	-0.000205268051003536	0.002231458269685	0.0238776489054873	2776034.58660941	0.0239585304078598	1.02411469142812	-0.000205268051003536	1.02379764625343	0.0254464022021758	0	0.0237529252505827	0.0239493905862214	2776034.58667024	0.0235871481454047	0.0237783864124189	2776034.5862778	0.0234877112071948
ethHispanic:famChild	-0.0238165315451206	-0.0237715473888845	-0.223777638581397	-0.0230511871237024	-1.02413235518908	-0.023191391522418	-0.023433388655679	-0.0235507325572539	-4.03088988333006E-05	-0.000122702251291008	0.0235975600548969	0.0234589470915579	0.223600325177085	1.02384293304659	-4.03088988333006E-05	27656301.0454824	0.0231315449787933	0	0.222935669886477	0.0236509061131209	0.0235794814349779	0.223717871325324	0.0234689248147195	0.023755230973295	0.223702771103353
ethBlack:famTwin	-0.190130381503623	-0.0234052052999598	-0.0233999424957336	-0.0290640218005166	-0.019824787967461	-1.03065789841987	-0.0258671400374308	-0.0258196381038334	0.000777672635218292	-0.00534909868135995	0.192051357036436	0.0252237394017576	0.024785467926547	0.189351128638998	0.000777672635218292	0.0231315449787933	536484287.621405	0	1.02944605686477	0.190925174595078	0.0246300627025719	0.0250051392080098	0.193140902078007	0.0219798201275961	0.0249889025324745
ethAsian:famTwin	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
ethHispanic:famTwin	-0.0232010584696114	-0.0240387707656515	-0.223601144766981	-0.0265801278647533	-0.0207864581296927	-1.02729018632717	-0.0244515600249059	-0.0245280303417383	0.000616274373564922	-0.000296504219099567	0.0245953362800284	0.0250557367097241	0.224453588118488	0.0232592877699353	0.000616274373564922	0.222935669886477	1.02944605686477	0	234021934.993277	0.0239582224335466	0.0245103801053452	0.224230755715213	0.0251119480568196	0.0239014403925007	0.224548400147941
ethBlack:famSibling	-0.190747726705487	-0.0236110008310505	-0.0239250194373203	-0.0245339728562016	-0.0244398779038476	-0.0238941784465175	-0.0495008143165027	-0.0241941326313094	-0.000164805987751014	-0.000650651023982381	0.190719242042659	0.024094026959306	0.0240096334900731	0.190601161706669	-0.000164805987751014	0.0236509061131209	0.190925174595078	0	0.0239582224335466	0.549126529263364	0.0489352283821505	0.0489172223887007	0.190797390868365	0.0236464109656927	0.0237411300930305
ethAsian:famSibling	-0.0237227293836337	-2776034.5866118	-0.023801014687455	-0.0247176644360182	-0.0233200894472744	-0.0243571582473462	-0.0491843526099389	-0.0240633404940337	8.14979501336502E-05	0.000314268952068898	0.0240756536327409	2776034.58712189	0.024031499303689	0.023740903059757	8.14979501336502E-05	0.0235794814349779	0.0246300627025719	0	0.0245103801053452	0.0489352283821505	2776034.94522617	0.0488719122910716	0.024143425271103	2776034.58656909	0.0239176713584075
ethHispanic:famSibling	-0.0238509353218037	-0.0237364359515111	-0.223813013498558	-0.0242234510144319	-0.0236168646444364	-0.0244530227229242	-0.0489881758273534	-0.0239708466462066	4.56138907343494E-05	-0.00141223691620152	0.0239985854006052	0.0239570786920573	0.22391408434107	0.0236703449794517	4.56138907343494E-05	0.223717871325324	0.0250051392080098	0	0.224230755715213	0.0489172223887007	0.0488719122910716	0.499023648963336	0.0241602283796845	0.0236061177537507	0.223962169254673
ethBlack:famOther	-0.190543539419183	-0.0236701519763141	-0.0238263306272248	-0.0253119509898178	-0.0233180123120797	-0.0252047524365899	-0.0245653620210343	-0.357703907374572	8.1919139857354E-05	-0.00173540146186277	0.191001954079834	0.0244546720071311	0.0241756199051724	0.190300753822968	8.1919139857354E-05	0.0234689248147195	0.193140902078007	0	0.0251119480568196	0.190797390868365	0.024143425271103	0.0241602283796845	4997024.13708618	0.356790641386149	0.357408557575104
ethAsian:famOther	-0.0235982539171211	-2776034.58679018	-0.023789277122574	-0.0232965565986291	-0.0235839907800947	-0.023290926232266	-0.0233673440735018	-0.356812464911102	5.99870523943372E-05	0.00158139474718642	0.0235903636163333	2776034.58655996	0.0237168621460648	0.0238345023166807	5.99870523943372E-05	0.023755230973295	0.0219798201275961	0	0.0239014403925007	0.0236464109656927	2776034.58656909	0.0236061177537507	0.356790641386149	20255750.0721761	0.35708518311905
ethHispanic:famOther	-0.023603936601252	-0.0239230334423816	-0.223726741307308	-0.024071231298798	-0.0228839936455257	-0.0246502916961579	-0.0236162617067214	-0.357118256510536	0.000213025091058376	-0.00068275005754803	0.0239408946123176	0.023972746891028	0.22386607780078	0.0235737583740995	0.000213025091058376	0.223702771103353	0.0249889025324745	0	0.224548400147941	0.0237411300930305	0.0239176713584075	0.223962169254673	0.357408557575104	0.35708518311905	1.55741965700759
"""

# Convert the data string into a pandas DataFrame
data = StringIO(data_str)
df = pd.read_csv(data, delim_whitespace=True)

# Print the DataFrame
print(df)


# Save the DataFrame as a CSV file
df.to_csv('/users/d/desktop/output.csv', index=False)


                        ethBlack      ethAsian  ethHispanic  famParent  \
ethBlack                0.190895  2.355584e-02     0.023965   0.023907   
ethAsian                0.023556  2.776035e+06     0.023736   0.023485   
ethHispanic             0.023965  2.373567e-02     0.223885   0.023841   
famParent               0.023907  2.348547e-02     0.023841   0.047315   
famChild                0.025059  2.310707e-02     0.024308   0.022953   
famTwin                 0.023316  2.388884e-02     0.023547   0.026287   
famSibling              0.024357  2.333548e-02     0.024014   0.025987   
famOther                0.024066  2.354460e-02     0.023889   0.025296   
don_age                 0.000291 -1.635642e-04     0.000117  -0.000075   
don_gender.L            0.000757 -8.129577e-04     0.000131   0.000658   
ethBlack:famParent     -0.190533 -2.370038e-02    -0.023830  -0.044524   
ethAsian:famParent     -0.023703 -2.776035e+06    -0.023809  -0.045115   
ethHispanic:famParent  -0.023862 -2.37

In [14]:
df.head(10)

,ethBlack,ethAsian,ethHispanic,famParent,famChild,famTwin,famSibling,famOther,don_age,don_gender.L,...,ethHispanic:famChild,ethBlack:famTwin,ethAsian:famTwin,ethHispanic:famTwin,ethBlack:famSibling,ethAsian:famSibling,ethHispanic:famSibling,ethBlack:famOther,ethAsian:famOther,ethHispanic:famOther
ethBlack,0.190895,2.355584e-02,0.023965,0.023907,0.025059,0.023316,0.024357,0.024066,0.000291,0.000757,...,-0.023817,-0.190130,0,-0.023201,-0.190748,-2.372273e-02,-0.023851,-0.190544,-2.359825e-02,-0.023604
ethAsian,0.023556,2.776035e+06,0.023736,0.023485,0.023107,0.023889,0.023335,0.023545,-0.000164,-0.000813,...,-0.023772,-0.023405,0,-0.024039,-0.023611,-2.776035e+06,-0.023736,-0.023670,-2.776035e+06,-0.023923
ethHispanic,0.023965,2.373567e-02,0.223885,0.023841,0.024308,0.023547,0.024014,0.023889,0.000117,0.000131,...,-0.223778,-0.023400,0,-0.223601,-0.023925,-2.380101e-02,-0.223813,-0.023826,-2.378928e-02,-0.223727
famParent,0.023907,2.348547e-02,0.023841,0.047315,0.022953,0.026287,0.025987,0.025296,-0.000075,0.000658,...,-0.023051,-0.029064,0,-0.026580,-0.024534,-2.471766e-02,-0.024223,-0.025312,-2.329656e-02,-0.024071
famChild,0.025059,2.310707e-02,0.024308,0.022953,1.028315,0.020846,0.024964,0.024135,0.001012,0.000766,...,-1.024132,-0.019825,0,-0.020786,-0.024440,-2.332009e-02,-0.023617,-0.023318,-2.358399e-02,-0.022884
famTwin,0.023316,2.388884e-02,0.023547,0.026287,0.020846,1.027655,0.024323,0.024527,-0.000623,0.002511,...,-0.023191,-1.030658,0,-1.027290,-0.023894,-2.435716e-02,-0.024453,-0.025205,-2.329093e-02,-0.024650
famSibling,0.024357,2.333548e-02,0.024014,0.025987,0.024964,0.024323,0.050672,0.024918,0.000342,0.000550,...,-0.023433,-0.025867,0,-0.024452,-0.049501,-4.918435e-02,-0.048988,-0.024565,-2.336734e-02,-0.023616
famOther,0.024066,2.354460e-02,0.023889,0.025296,0.024135,0.024527,0.024918,0.357856,0.000125,0.000532,...,-0.023551,-0.025820,0,-0.024528,-0.024194,-2.406334e-02,-0.023971,-0.357704,-3.568125e-01,-0.357118
don_age,0.000291,-1.635642e-04,0.000117,-0.000075,0.001012,-0.000623,0.000342,0.000125,0.000234,0.000056,...,-0.000040,0.000778,0,0.000616,-0.000165,8.149795e-05,0.000046,0.000082,5.998705e-05,0.000213
don_gender.L,0.000757,-8.129577e-04,0.000131,0.000658,0.000766,0.002511,0.000550,0.000532,0.000056,0.012138,...,-0.000123,-0.005349,0,-0.000297,-0.000651,3.142690e-04,-0.001412,-0.001735,1.581395e-03,-0.000683
